In [ ]:
#default_exp ringutils

# Meta

> This module deals with the metadata I have received from collaborators.

> It defines the location of ring resonances for the RingCube plotting.


In [ ]:
#export
import pandas as pd
import pkg_resources as pr
from astropy.time import Time


def get_order(name):
    ratio = name.split()[1]
    a, b = ratio.split(":")
    return int(a) - int(b)


def get_resonances():
    with pr.resource_stream("pyciss2", "data/ring_resonances.csv") as f:
        resonances = pd.read_csv(f)
    resonances.columns = ["name", "radius", "a_moon", "n", "kappa"]
    resonances = resonances.sort_values(by="radius", ascending=True)
    resonances["order"] = resonances.name.map(get_order)
    return resonances

`get_resonances` returns a pandas.DataFrame with the location of resonances in the Saturn rings.

In [ ]:
get_resonances()

,name,radius,a_moon,n,kappa,order
17,Titan 2:0,64683.367989,1.221766e+06,0.000005,0.000005,2
1026,Pan 3:1,65653.446755,1.335845e+05,0.000126,0.000126,2
1336,Daphnis 3:1,67023.465716,1.365056e+05,0.000122,0.000122,2
138,Atlas 3:1,67568.098552,1.376659e+05,0.000121,0.000120,2
434,Prometheus 3:1,68372.234208,1.393780e+05,0.000119,0.000118,2
...,...,...,...,...,...,...
11,Mimas 9:6,141805.194391,1.855360e+05,0.000077,0.000077,3
8,Mimas 6:4,141826.993119,1.855360e+05,0.000077,0.000077,2
134,Epimetheus 32:29,141860.874828,1.514144e+05,0.000105,0.000104,3
3,Mimas 3:2,141892.207674,1.855360e+05,0.000077,0.000077,1


In [ ]:
#export
def get_prime_resonances():
    resonances = get_resonances()
    prime_resonances = resonances[resonances.order == 1].drop("order", axis=1)
    # filter out Janus and Epimetheus as we have a more precise file for that.
    prime_resonances = prime_resonances.loc[
        ~prime_resonances.name.str.startswith("Janus")
    ]
    prime_resonances = prime_resonances.loc[
        ~prime_resonances.name.str.startswith("Epimetheus")
    ]
    return prime_resonances

`get_prime_resonances()` filters the resonances down to only return prime resonances.

In [ ]:
get_prime_resonances()

,name,radius,a_moon,n,kappa
16,Titan 1:0,77846.647183,1.221766e+06,0.000005,0.000005
18,Hyperion 1:0,84196.651805,1.480799e+06,0.000003,0.000003
1025,Pan 2:1,85105.788697,1.335845e+05,0.000126,0.000126
1335,Daphnis 2:1,86924.049609,1.365056e+05,0.000122,0.000122
137,Atlas 2:1,87646.535717,1.376659e+05,0.000121,0.000120
...,...,...,...,...,...
1013,Pandora 97:96,140744.374617,1.417126e+05,0.000116,0.000115
1016,Pandora 98:97,140754.270526,1.417126e+05,0.000116,0.000115
1019,Pandora 99:98,140763.966193,1.417126e+05,0.000116,0.000115
1022,Pandora 100:99,140773.467637,1.417126e+05,0.000116,0.000115


In [ ]:
#export
def get_janus_epimetheus_resonances():
    "Filter resonances for only Janus and Epithemeus resonances."
    w = [len("              Janus1"), len(" reson"), len("  Resonance radius R")]

    def get_janos_epi_order(reso):
        a, b = reso.split(":")
        return int(a) - int(b)

    fname = pr.resource_filename("pyciss2", "data/ring_janus_epimetheus_resonances.txt")
    with open(fname) as f:
        jan_epi_resonances = pd.read_fwf(
            f, skiprows=15, header=0, widths=w, skipfooter=1
        )

    # replace column names
    jan_epi_resonances.columns = ["moon", "reson", "radius"]

    # calculate order from resonance name
    jan_epi_resonances["order"] = jan_epi_resonances.reson.map(get_janos_epi_order)

    def func(x):
        "Remove space from resonce string"
        return ":".join(i.strip() for i in x.split(":"))

    jan_epi_resonances.reson = jan_epi_resonances.reson.map(func)

    # calculate name for axes display
    jan_epi_resonances["name"] = (
        jan_epi_resonances.moon + " " + jan_epi_resonances.reson
    )
    return jan_epi_resonances

In [ ]:
get_janus_epimetheus_resonances()

,moon,reson,radius,order,name
0,Janus1,2:1,96235.150437,1,Janus1 2:1
1,Janus1,3:1,74051.968798,2,Janus1 3:1
2,Janus1,3:2,115943.966750,1,Janus1 3:2
3,Janus1,4:2,95979.967411,2,Janus1 4:2
4,Janus1,4:3,125250.859597,1,Janus1 4:3
...,...,...,...,...,...
71,Epimetheus2,11:9,132590.476835,2,Epimetheus2 11:9
72,Epimetheus2,12:10,134207.825664,2,Epimetheus2 12:10
73,Epimetheus2,13:11,135568.924275,2,Epimetheus2 13:11
74,Epimetheus2,14:12,136730.263366,2,Epimetheus2 14:12


In [ ]:
#export
def get_prime_jan_epi():
    jan_epi_resonances = get_janus_epimetheus_resonances()
    # remove orders > 1 and drop unrequired columns
    prime_jan_epis = jan_epi_resonances[jan_epi_resonances.order == 1]
    to_drop = ["order", "moon"]
    prime_jan_epis = prime_jan_epis.drop(to_drop, axis=1)
    return prime_jan_epis

In [ ]:
get_prime_jan_epi()

,reson,radius,name
0,2:1,96235.150437,Janus1 2:1
2,3:2,115943.966750,Janus1 3:2
4,4:3,125250.859597,Janus1 4:3
6,5:4,130683.741757,Janus1 5:4
8,6:5,134247.454656,Janus1 6:5
10,7:6,136765.832643,Janus1 7:6
19,2:1,96265.101223,Epimetheus1 2:1
21,3:2,115980.464512,Epimetheus1 3:2
23,4:3,125290.393760,Epimetheus1 4:3
25,5:4,130725.038596,Epimetheus1 5:4


In [ ]:
#export
def get_all_prime_resonances():
    prime_resonances = get_prime_resonances()
    prime_jan_epis = get_prime_jan_epi()
    all_resonances = pd.concat(
        [prime_resonances, prime_jan_epis], ignore_index=True, sort=False
    )
    all_resonances.sort_values(by="radius", inplace=True)
    all_resonances["moon"] = all_resonances.name.map(lambda x: x.split()[0].lower())
    return all_resonances

In [ ]:
get_all_prime_resonances()

,name,radius,a_moon,n,kappa,reson,moon
0,Titan 1:0,77846.647183,1.221766e+06,0.000005,0.000005,NaN,titan
1,Hyperion 1:0,84196.651805,1.480799e+06,0.000003,0.000003,NaN,hyperion
2,Pan 2:1,85105.788697,1.335845e+05,0.000126,0.000126,NaN,pan
3,Daphnis 2:1,86924.049609,1.365056e+05,0.000122,0.000122,NaN,daphnis
4,Atlas 2:1,87646.535717,1.376659e+05,0.000121,0.000120,NaN,atlas
...,...,...,...,...,...,...,...
497,Pandora 97:96,140744.374617,1.417126e+05,0.000116,0.000115,NaN,pandora
498,Pandora 98:97,140754.270526,1.417126e+05,0.000116,0.000115,NaN,pandora
499,Pandora 99:98,140763.966193,1.417126e+05,0.000116,0.000115,NaN,pandora
500,Pandora 100:99,140773.467637,1.417126e+05,0.000116,0.000115,NaN,pandora


In [ ]:
#export
def which_epi_janus_resonance(name, time):
    """Find which swap situtation we are in by time.

    Starting from 2006-01-21 where a Janus-Epimetheus swap occured, and
    defining the next 4 years until the next swap as `scenario1, and the 4
    years after that `scenario2`.
    Calculate in units of 4 years, in which scenario the given time falls.

    Parameters
    ----------
    time : timestring, datetime
        Time of the image. The astropy Time object can deal with both formats.

    Returns
    -------
    str
        The given name string (either `janus` or `epimetheus`) and attach
        a 1 or 2, as appropriate.
    """
    t1 = Time('2002-01-21').to_datetime()
    delta = Time(time).to_datetime() - t1
    yearfraction = delta.days / 365
    if int(yearfraction / 4) % 2 == 0:
        return name + '2'
    else:
        return name + '1'

In [ ]:
which_epi_janus_resonance('janus', '2008-01-01')

'janus1'

In [ ]:
which_epi_janus_resonance('janus', '2012-01-01')

'janus2'

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_io.ipynb.
Converted 01_opusapi.ipynb.
Converted 02_pipeline.ipynb.
Converted 03_ringutils.ipynb.
Converted 04_plotting.ipynb.
Converted 05_index.ipynb.
Converted 06_ringcube.ipynb.
Converted index.ipynb.
Converted to_be_implemented.ipynb.
